In [4]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD001"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "lgb_only_pgts"

# Fixed model hyperparameters (as requested)
LGB_PARAMS = dict(
    n_estimators=500,
    learning_rate=0.1,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=-1,      # silence LightGBM warnings
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import clone
import lightgbm as lgb

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = preprocessing.MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score is renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# LightGBM model builder
# ============================================
def build_lgb():
    return lgb.LGBMRegressor(**LGB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_lgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_LGB")
    return summary

# ============================================
# Random 80/20 holdout (baseline)
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
lgb_hold = build_lgb().fit(X_tr, y_tr)
p_hold = lgb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}.csv")

# ============================================
# PGTS main (Embargo=10)
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_lgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_LGB")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries)
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — LightGBM]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main
#report_lines.append("\n[PGTS — LightGBM (Embargo=10)]")
#report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) - LightGBM, Embargo=10]")
report_lines.append(fmt_summary("Results :", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] - LightGBM (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results:",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results:", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}.txt", final_report)


Dataset=FD001 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — LightGBM]
  R2: 0.9894 | MSE: 48.2539 | MAE: 5.0370 | RUL Score: 3236.1059
[PGTS — Null Model (label permutation) - LightGBM, Embargo=10]
Results :
  R2 (mean±std):        -0.3631 ± 0.3911
  MSE (mean±std):       4995.0485 ± 2816.6261
  MAE (mean±std):       57.4197 ± 15.1311
  RUL Score (mean±std): 125399549003.4736 ± 1207679375425.8418
  Folds:                400

[PGTS — Embargo Sensitivity] - LightGBM (Embargo 0 vs 10)
Embargo=0 Results:
  R2 (mean±std):        -13.4234 ± 8.0440
  MSE (mean±std):       8221.2526 ± 6482.7785
  MAE (mean±std):       80.5886 ± 31.0111
  RUL Score (mean±std): 2669292984012.6484 ± 45172488807766.0547
  Folds:                400

Embargo=10 Results:
  R2 (mean±std):        -27.5141 ± 33.1033
  MSE (mean±std):       8857.2287 ± 6713.8486
  MAE (mean±std):       85.0832 ± 31.1085
  RUL Score (mean±std): 26692929762

In [5]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD002"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "lgb_only_pgts"

# Fixed model hyperparameters (as requested)
LGB_PARAMS = dict(
    n_estimators=500,
    learning_rate=0.1,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=-1,      # silence LightGBM warnings
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import clone
import lightgbm as lgb

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = preprocessing.MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score is renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# LightGBM model builder
# ============================================
def build_lgb():
    return lgb.LGBMRegressor(**LGB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_lgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_LGB")
    return summary

# ============================================
# Random 80/20 holdout (baseline)
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
lgb_hold = build_lgb().fit(X_tr, y_tr)
p_hold = lgb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}.csv")

# ============================================
# PGTS main (Embargo=10)
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_lgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_LGB")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries)
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — LightGBM]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main
#report_lines.append("\n[PGTS — LightGBM (Embargo=10)]")
#report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) - LightGBM, Embargo=10]")
report_lines.append(fmt_summary("Results :", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] - LightGBM (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results:",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results:", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}.txt", final_report)


Dataset=FD002 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — LightGBM]
  R2: 0.9832 | MSE: 79.0024 | MAE: 6.7962 | RUL Score: 13277.0178
[PGTS — Null Model (label permutation) - LightGBM, Embargo=10]
Results :
  R2 (mean±std):        -0.4369 ± 0.4464
  MSE (mean±std):       5347.5968 ± 3214.0103
  MAE (mean±std):       58.9451 ± 16.3755
  RUL Score (mean±std): 13576101086265.7461 ± 345108954178335.0625
  Folds:                1040

[PGTS — Embargo Sensitivity] - LightGBM (Embargo 0 vs 10)
Embargo=0 Results:
  R2 (mean±std):        -23.4150 ± 20.5719
  MSE (mean±std):       10683.7284 ± 6361.2987
  MAE (mean±std):       94.1182 ± 27.6772
  RUL Score (mean±std): 17857124866170.3828 ± 457822180716058.5000
  Folds:                1040

Embargo=10 Results:
  R2 (mean±std):        -48.4105 ± 68.1574
  MSE (mean±std):       11443.1721 ± 6588.4857
  MAE (mean±std):       98.5619 ± 28.0586
  RUL Score (mean±std):

In [6]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD003"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "lgb_only_pgts"

# Fixed model hyperparameters (as requested)
LGB_PARAMS = dict(
    n_estimators=500,
    learning_rate=0.1,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=-1,      # silence LightGBM warnings
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import clone
import lightgbm as lgb

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = preprocessing.MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score is renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# LightGBM model builder
# ============================================
def build_lgb():
    return lgb.LGBMRegressor(**LGB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_lgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_LGB")
    return summary

# ============================================
# Random 80/20 holdout (baseline)
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
lgb_hold = build_lgb().fit(X_tr, y_tr)
p_hold = lgb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}.csv")

# ============================================
# PGTS main (Embargo=10)
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_lgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_LGB")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries)
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — LightGBM]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main
#report_lines.append("\n[PGTS — LightGBM (Embargo=10)]")
#report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) - LightGBM, Embargo=10]")
report_lines.append(fmt_summary("Results :", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] - LightGBM (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results:",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results:", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}.txt", final_report)


Dataset=FD003 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — LightGBM]
  R2: 0.9898 | MSE: 99.7244 | MAE: 6.9449 | RUL Score: 15041.6702
[PGTS — Null Model (label permutation) - LightGBM, Embargo=10]
Results :
  R2 (mean±std):        -0.3849 ± 0.4397
  MSE (mean±std):       7803.1683 ± 6243.0655
  MAE (mean±std):       69.7110 ± 25.2408
  RUL Score (mean±std): 43056464140658896.0000 ± 610550778194821632.0000
  Folds:                400

[PGTS — Embargo Sensitivity] - LightGBM (Embargo 0 vs 10)
Embargo=0 Results:
  R2 (mean±std):        -12.2814 ± 6.5595
  MSE (mean±std):       12152.0036 ± 12669.2231
  MAE (mean±std):       94.5143 ± 43.5410
  RUL Score (mean±std): 9771996031213809664.0000 ± 177496650878245535744.0000
  Folds:                400

Embargo=10 Results:
  R2 (mean±std):        -22.4704 ± 21.9772
  MSE (mean±std):       12907.0482 ± 12988.0567
  MAE (mean±std):       99.1431 ± 43.6701
  RUL S

In [7]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD004"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "lgb_only_pgts"

# Fixed model hyperparameters (as requested)
LGB_PARAMS = dict(
    n_estimators=500,
    learning_rate=0.1,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=-1,      # silence LightGBM warnings
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import clone
import lightgbm as lgb

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = preprocessing.MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score is renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# LightGBM model builder
# ============================================
def build_lgb():
    return lgb.LGBMRegressor(**LGB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_lgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_LGB")
    return summary

# ============================================
# Random 80/20 holdout (baseline)
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
lgb_hold = build_lgb().fit(X_tr, y_tr)
p_hold = lgb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}.csv")

# ============================================
# PGTS main (Embargo=10)
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_lgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_LGB")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries)
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — LightGBM]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main
#report_lines.append("\n[PGTS — LightGBM (Embargo=10)]")
#report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) - LightGBM, Embargo=10]")
report_lines.append(fmt_summary("Results :", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] - LightGBM (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results:",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results:", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}.txt", final_report)


Dataset=FD004 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — LightGBM]
  R2: 0.9830 | MSE: 136.9355 | MAE: 8.8780 | RUL Score: 25974.5151
[PGTS — Null Model (label permutation) - LightGBM, Embargo=10]
Results :
  R2 (mean±std):        -0.4305 ± 0.3565
  MSE (mean±std):       7874.3636 ± 5559.4215
  MAE (mean±std):       70.2189 ± 23.2377
  RUL Score (mean±std): 540974661719257728.0000 ± 12406418392449462272.0000
  Folds:                996

[PGTS — Embargo Sensitivity] - LightGBM (Embargo 0 vs 10)
Embargo=0 Results:
  R2 (mean±std):        -19.4229 ± 16.6162
  MSE (mean±std):       14481.7819 ± 11311.7217
  MAE (mean±std):       106.6296 ± 39.0857
  RUL Score (mean±std): 25807753653300912128.0000 ± 558482759989211430912.0000
  Folds:                996

Embargo=10 Results:
  R2 (mean±std):        -36.8880 ± 53.0221
  MSE (mean±std):       15354.4717 ± 11603.3629
  MAE (mean±std):       111.2491 ± 39.2876

In [9]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD001"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "catboost_only_pgts"

# Fixed model hyperparameters (CatBoost)
CB_PARAMS = dict(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=0,         # silence CatBoost training logs
    loss_function='RMSE'
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from catboost import CatBoostRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score is renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# CatBoost model builder
# ============================================
def build_cb():
    return CatBoostRegressor(**CB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_cb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_CatBoost")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — CatBoost
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
cb_hold = build_cb().fit(X_tr, y_tr)
p_hold = cb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_CatBoost.csv")

# ============================================
# PGTS main (Embargo=10) — CatBoost
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — CatBoost
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_cb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_CatBoost")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — CatBoost
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — CatBoost
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — CatBoost]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main
#report_lines.append("\n[PGTS — CatBoost (Embargo=10)]")
#report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — CatBoost, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — CatBoost (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_CatBoost.txt", final_report)


Dataset=FD001 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — CatBoost]
  R2: 0.9872 | MSE: 58.3519 | MAE: 5.8927 | RUL Score: 3761.2999
[PGTS — Null Model (label permutation) — CatBoost, Embargo=10]
Results
  R2 (mean±std):        -0.1523 ± 0.1833
  MSE (mean±std):       4244.0547 ± 2322.6343
  MAE (mean±std):       54.0617 ± 13.5151
  RUL Score (mean±std): 3882929923.2324 ± 44450061816.1704
  Folds:                400

[PGTS — Embargo Sensitivity] — CatBoost (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -19.6719 ± 14.2012
  MSE (mean±std):       9950.8117 ± 6364.9369
  MAE (mean±std):       91.3019 ± 27.6341
  RUL Score (mean±std): 3092642413689.4863 ± 49818096206064.9766
  Folds:                400

Embargo=10 Results
  R2 (mean±std):        -41.4518 ± 54.6701
  MSE (mean±std):       10765.0992 ± 6539.1773
  MAE (mean±std):       96.4234 ± 27.4402
  RUL Score (mean±std): 3092642409465.341

In [10]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD002"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "catboost_only_pgts"

# Fixed model hyperparameters (CatBoost)
CB_PARAMS = dict(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=0,         # silence CatBoost training logs
    loss_function='RMSE'
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from catboost import CatBoostRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score is renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# CatBoost model builder
# ============================================
def build_cb():
    return CatBoostRegressor(**CB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_cb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_CatBoost")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — CatBoost
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
cb_hold = build_cb().fit(X_tr, y_tr)
p_hold = cb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_CatBoost.csv")

# ============================================
# PGTS main (Embargo=10) — CatBoost
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — CatBoost
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_cb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_CatBoost")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — CatBoost
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — CatBoost
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — CatBoost]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main
#report_lines.append("\n[PGTS — CatBoost (Embargo=10)]")
#report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — CatBoost, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — CatBoost (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_CatBoost.txt", final_report)


Dataset=FD002 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — CatBoost]
  R2: 0.9498 | MSE: 235.6130 | MAE: 12.0735 | RUL Score: 39339.4042
[PGTS — Null Model (label permutation) — CatBoost, Embargo=10]
Results
  R2 (mean±std):        -0.3717 ± 0.2716
  MSE (mean±std):       5028.9107 ± 2620.1954
  MAE (mean±std):       57.6963 ± 14.2238
  RUL Score (mean±std): 326837721045.9213 ± 9513441848210.2676
  Folds:                1040

[PGTS — Embargo Sensitivity] — CatBoost (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -22.7586 ± 18.4219
  MSE (mean±std):       10625.4708 ± 6343.3908
  MAE (mean±std):       94.8369 ± 26.6789
  RUL Score (mean±std): 15485141355664.2148 ± 348992854105390.0625
  Folds:                1040

Embargo=10 Results
  R2 (mean±std):        -47.1226 ± 62.5902
  MSE (mean±std):       11420.2588 ± 6533.8169
  MAE (mean±std):       99.5583 ± 26.7327
  RUL Score (mean±std): 15485

In [11]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD003"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "catboost_only_pgts"

# Fixed model hyperparameters (CatBoost)
CB_PARAMS = dict(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=0,         # silence CatBoost training logs
    loss_function='RMSE'
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from catboost import CatBoostRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score is renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# CatBoost model builder
# ============================================
def build_cb():
    return CatBoostRegressor(**CB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_cb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_CatBoost")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — CatBoost
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
cb_hold = build_cb().fit(X_tr, y_tr)
p_hold = cb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_CatBoost.csv")

# ============================================
# PGTS main (Embargo=10) — CatBoost
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — CatBoost
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_cb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_CatBoost")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — CatBoost
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — CatBoost
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — CatBoost]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main
#report_lines.append("\n[PGTS — CatBoost (Embargo=10)]")
#report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — CatBoost, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — CatBoost (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_CatBoost.txt", final_report)


Dataset=FD003 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — CatBoost]
  R2: 0.9854 | MSE: 143.2805 | MAE: 8.6236 | RUL Score: 21036.0152
[PGTS — Null Model (label permutation) — CatBoost, Embargo=10]
Results
  R2 (mean±std):        -0.1694 ± 0.2336
  MSE (mean±std):       6644.2492 ± 5324.2171
  MAE (mean±std):       65.5176 ± 23.4308
  RUL Score (mean±std): 48685974422202.9766 ± 617439568807418.0000
  Folds:                400

[PGTS — Embargo Sensitivity] — CatBoost (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -16.8319 ± 11.9293
  MSE (mean±std):       13920.0070 ± 12967.4665
  MAE (mean±std):       103.7241 ± 41.2806
  RUL Score (mean±std): 19091951148569608192.0000 ± 368981738485275164672.0000
  Folds:                400

Embargo=10 Results
  R2 (mean±std):        -32.1086 ± 40.2939
  MSE (mean±std):       14835.3068 ± 13253.6739
  MAE (mean±std):       108.9007 ± 41.1822
  RUL Score 

In [12]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD004"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "catboost_only_pgts"

# Fixed model hyperparameters (CatBoost)
CB_PARAMS = dict(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=0,         # silence CatBoost training logs
    loss_function='RMSE'
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from catboost import CatBoostRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score is renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# CatBoost model builder
# ============================================
def build_cb():
    return CatBoostRegressor(**CB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_cb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_CatBoost")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — CatBoost
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
cb_hold = build_cb().fit(X_tr, y_tr)
p_hold = cb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_CatBoost.csv")

# ============================================
# PGTS main (Embargo=10) — CatBoost
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — CatBoost
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_cb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_CatBoost")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — CatBoost
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — CatBoost
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — CatBoost]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main
#report_lines.append("\n[PGTS — CatBoost (Embargo=10)]")
#report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — CatBoost, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — CatBoost (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_CatBoost.txt", final_report)


Dataset=FD004 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — CatBoost]
  R2: 0.9403 | MSE: 479.8289 | MAE: 16.9850 | RUL Score: 174405.5979
[PGTS — Null Model (label permutation) — CatBoost, Embargo=10]
Results
  R2 (mean±std):        -0.3537 ± 0.2555
  MSE (mean±std):       7196.4212 ± 4513.8248
  MAE (mean±std):       68.1682 ± 20.4050
  RUL Score (mean±std): 239274966969255.1875 ± 4103275332667334.0000
  Folds:                996

[PGTS — Embargo Sensitivity] — CatBoost (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -19.1938 ± 15.4521
  MSE (mean±std):       14408.4218 ± 11265.4815
  MAE (mean±std):       107.3575 ± 38.1147
  RUL Score (mean±std): 36269861794884620288.0000 ± 1115284983045348524032.0000
  Folds:                996

Embargo=10 Results
  R2 (mean±std):        -36.7452 ± 51.3636
  MSE (mean±std):       15311.1655 ± 11532.9710
  MAE (mean±std):       112.2038 ± 38.0998
  RUL S

In [13]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD001"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "gb_only_pgts"

# Fixed model hyperparameters (Gradient Boosting)
GB_PARAMS = dict(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score = renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# Gradient Boosting model builder
# ============================================
def build_gb():
    return GradientBoostingRegressor(**GB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_gb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_GradientBoosting")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — Gradient Boosting
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
gb_hold = build_gb().fit(X_tr, y_tr)
p_hold = gb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_GradientBoosting.csv")

# ============================================
# PGTS main (Embargo=10) — Gradient Boosting
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — Gradient Boosting
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_gb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_GradientBoosting")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — Gradient Boosting
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — Gradient Boosting
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — GradientBoosting]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main (aynı CatBoost örneğinde olduğu gibi satırı saklı tutabilir veya açabilirsin)
# report_lines.append("\n[PGTS — GradientBoosting (Embargo=10)]")
# report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — GradientBoosting, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — GradientBoosting (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_GradientBoosting.txt", final_report)


Dataset=FD001 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — GradientBoosting]
  R2: 0.9840 | MSE: 72.9487 | MAE: 6.0383 | RUL Score: 5748.0022
[PGTS — Null Model (label permutation) — GradientBoosting, Embargo=10]
Results
  R2 (mean±std):        -0.2952 ± 0.2805
  MSE (mean±std):       4697.7835 ± 2471.0137
  MAE (mean±std):       56.1931 ± 13.7121
  RUL Score (mean±std): 2754605875.8714 ± 29687775830.3209
  Folds:                400

[PGTS — Embargo Sensitivity] — GradientBoosting (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -16.8471 ± 10.6445
  MSE (mean±std):       9275.6666 ± 6472.0449
  MAE (mean±std):       87.2176 ± 28.9685
  RUL Score (mean±std): 3398934526074.6626 ± 57268855973195.1016
  Folds:                400

Embargo=10 Results
  R2 (mean±std):        -34.6057 ± 41.9510
  MSE (mean±std):       10018.2798 ± 6680.5703
  MAE (mean±std):       92.0705 ± 28.9645
  RUL Score (mean

In [14]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD002"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "gb_only_pgts"

# Fixed model hyperparameters (Gradient Boosting)
GB_PARAMS = dict(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score = renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# Gradient Boosting model builder
# ============================================
def build_gb():
    return GradientBoostingRegressor(**GB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_gb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_GradientBoosting")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — Gradient Boosting
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
gb_hold = build_gb().fit(X_tr, y_tr)
p_hold = gb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_GradientBoosting.csv")

# ============================================
# PGTS main (Embargo=10) — Gradient Boosting
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — Gradient Boosting
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_gb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_GradientBoosting")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — Gradient Boosting
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — Gradient Boosting
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — GradientBoosting]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main (aynı CatBoost örneğinde olduğu gibi satırı saklı tutabilir veya açabilirsin)
# report_lines.append("\n[PGTS — GradientBoosting (Embargo=10)]")
# report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — GradientBoosting, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — GradientBoosting (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_GradientBoosting.txt", final_report)


Dataset=FD002 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — GradientBoosting]
  R2: 0.9804 | MSE: 91.7819 | MAE: 7.3138 | RUL Score: 15072.2448
[PGTS — Null Model (label permutation) — GradientBoosting, Embargo=10]
Results
  R2 (mean±std):        -0.4315 ± 0.2857
  MSE (mean±std):       5245.0847 ± 2734.3053
  MAE (mean±std):       58.7280 ± 14.5352
  RUL Score (mean±std): 129150615916.7789 ± 1918472227342.8181
  Folds:                1040

[PGTS — Embargo Sensitivity] — GradientBoosting (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -22.9671 ± 18.7117
  MSE (mean±std):       10644.0295 ± 6321.2825
  MAE (mean±std):       95.0231 ± 26.4396
  RUL Score (mean±std): 12433577496370.4980 ± 247596363216614.0312
  Folds:                1040

Embargo=10 Results
  R2 (mean±std):        -47.5316 ± 62.9211
  MSE (mean±std):       11431.9680 ± 6516.2747
  MAE (mean±std):       99.6711 ± 26.5559
  RUL S

In [16]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD003"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "gb_only_pgts"

# Fixed model hyperparameters (Gradient Boosting)
GB_PARAMS = dict(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score = renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# Gradient Boosting model builder
# ============================================
def build_gb():
    return GradientBoostingRegressor(**GB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_gb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_GradientBoosting")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — Gradient Boosting
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
gb_hold = build_gb().fit(X_tr, y_tr)
p_hold = gb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_GradientBoosting.csv")

# ============================================
# PGTS main (Embargo=10) — Gradient Boosting
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — Gradient Boosting
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_gb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_GradientBoosting")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — Gradient Boosting
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — Gradient Boosting
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — GradientBoosting]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main (aynı CatBoost örneğinde olduğu gibi satırı saklı tutabilir veya açabilirsin)
# report_lines.append("\n[PGTS — GradientBoosting (Embargo=10)]")
# report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — GradientBoosting, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — GradientBoosting (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_GradientBoosting.txt", final_report)


Dataset=FD003 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — GradientBoosting]
  R2: 0.9866 | MSE: 130.9718 | MAE: 7.8019 | RUL Score: 28202.9870
[PGTS — Null Model (label permutation) — GradientBoosting, Embargo=10]
Results
  R2 (mean±std):        -0.3164 ± 0.3123
  MSE (mean±std):       7322.5549 ± 5771.3948
  MAE (mean±std):       68.1245 ± 23.5971
  RUL Score (mean±std): 2176135449317262.7500 ± 31034646256604252.0000
  Folds:                400

[PGTS — Embargo Sensitivity] — GradientBoosting (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -14.6885 ± 9.2831
  MSE (mean±std):       13292.9226 ± 13399.2947
  MAE (mean±std):       99.9149 ± 43.2716
  RUL Score (mean±std): 55770330263616946176.0000 ± 1077222843725632241664.0000
  Folds:                400

Embargo=10 Results
  R2 (mean±std):        -27.5954 ± 31.7693
  MSE (mean±std):       14147.5856 ± 13724.4535
  MAE (mean±std):       104.

In [17]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD004"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "gb_only_pgts"

# Fixed model hyperparameters (Gradient Boosting)
GB_PARAMS = dict(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score = renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# Gradient Boosting model builder
# ============================================
def build_gb():
    return GradientBoostingRegressor(**GB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_gb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_GradientBoosting")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — Gradient Boosting
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
gb_hold = build_gb().fit(X_tr, y_tr)
p_hold = gb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_GradientBoosting.csv")

# ============================================
# PGTS main (Embargo=10) — Gradient Boosting
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — Gradient Boosting
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_gb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_GradientBoosting")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — Gradient Boosting
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — Gradient Boosting
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — GradientBoosting]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main (aynı CatBoost örneğinde olduğu gibi satırı saklı tutabilir veya açabilirsin)
# report_lines.append("\n[PGTS — GradientBoosting (Embargo=10)]")
# report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — GradientBoosting, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — GradientBoosting (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_GradientBoosting.txt", final_report)


Dataset=FD004 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — GradientBoosting]
  R2: 0.9779 | MSE: 177.9456 | MAE: 10.0170 | RUL Score: 38718.4295
[PGTS — Null Model (label permutation) — GradientBoosting, Embargo=10]
Results
  R2 (mean±std):        -0.4035 ± 0.2781
  MSE (mean±std):       7455.2237 ± 4709.8973
  MAE (mean±std):       69.1501 ± 20.7402
  RUL Score (mean±std): 5383978822758562.0000 ± 85157925457180176.0000
  Folds:                996

[PGTS — Embargo Sensitivity] — GradientBoosting (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -19.6587 ± 15.3985
  MSE (mean±std):       14666.2917 ± 11303.1595
  MAE (mean±std):       108.2428 ± 38.1474
  RUL Score (mean±std): 24660465452448976896.0000 ± 740371961084914040832.0000
  Folds:                996

Embargo=10 Results
  R2 (mean±std):        -37.2196 ± 49.8725
  MSE (mean±std):       15578.6291 ± 11576.4453
  MAE (mean±std):       11

In [18]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD001"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "xgb_only_pgts"

# Fixed model hyperparameters (XGBoost)
XGB_PARAMS = dict(
    objective="reg:squarederror",
    learning_rate=0.2,
    max_depth=4,
    n_estimators=200,
    subsample=1.0,
    colsample_bytree=0.8,
    random_state=42
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score = renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# XGBoost model builder
# ============================================
def build_xgb():
    return XGBRegressor(**XGB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_xgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_XGBoost")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — XGBoost
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
xgb_hold = build_xgb().fit(X_tr, y_tr)
p_hold = xgb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_XGBoost.csv")

# ============================================
# PGTS main (Embargo=10) — XGBoost
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — XGBoost
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_xgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_XGBoost")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — XGBoost
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — XGBoost
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — XGBoost]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main (opsiyonel olarak açılabilir)
# report_lines.append("\n[PGTS — XGBoost (Embargo=10)]")
# report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — XGBoost, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — XGBoost (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_XGBoost.txt", final_report)


Dataset=FD001 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — XGBoost]
  R2: 0.9825 | MSE: 79.7548 | MAE: 6.8220 | RUL Score: 5054.8245
[PGTS — Null Model (label permutation) — XGBoost, Embargo=10]
Results
  R2 (mean±std):        -0.3206 ± 0.3580
  MSE (mean±std):       4833.7805 ± 2857.8303
  MAE (mean±std):       56.6898 ± 14.7574
  RUL Score (mean±std): 9891627813.1592 ± 91491181841.5948
  Folds:                400

[PGTS — Embargo Sensitivity] — XGBoost (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -18.3422 ± 13.5650
  MSE (mean±std):       9483.9062 ± 6340.7298
  MAE (mean±std):       88.6217 ± 27.9377
  RUL Score (mean±std): 3915047670640.4146 ± 60718162496540.4219
  Folds:                400

Embargo=10 Results
  R2 (mean±std):        -38.6473 ± 52.8056
  MSE (mean±std):       10254.4031 ± 6530.3002
  MAE (mean±std):       93.5969 ± 27.8659
  RUL Score (mean±std): 3915047662011.1392 ±

In [19]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD002"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "xgb_only_pgts"

# Fixed model hyperparameters (XGBoost)
XGB_PARAMS = dict(
    objective="reg:squarederror",
    learning_rate=0.2,
    max_depth=4,
    n_estimators=200,
    subsample=1.0,
    colsample_bytree=0.8,
    random_state=42
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score = renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# XGBoost model builder
# ============================================
def build_xgb():
    return XGBRegressor(**XGB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_xgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_XGBoost")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — XGBoost
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
xgb_hold = build_xgb().fit(X_tr, y_tr)
p_hold = xgb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_XGBoost.csv")

# ============================================
# PGTS main (Embargo=10) — XGBoost
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — XGBoost
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_xgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_XGBoost")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — XGBoost
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — XGBoost
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — XGBoost]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main (opsiyonel olarak açılabilir)
# report_lines.append("\n[PGTS — XGBoost (Embargo=10)]")
# report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — XGBoost, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — XGBoost (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_XGBoost.txt", final_report)


Dataset=FD002 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — XGBoost]
  R2: 0.9335 | MSE: 312.2766 | MAE: 13.0481 | RUL Score: 335032.3025
[PGTS — Null Model (label permutation) — XGBoost, Embargo=10]
Results
  R2 (mean±std):        -0.4679 ± 0.3038
  MSE (mean±std):       5386.7397 ± 2830.7087
  MAE (mean±std):       59.3320 ± 14.8057
  RUL Score (mean±std): 100880657707.9924 ± 1205031470677.7039
  Folds:                1040

[PGTS — Embargo Sensitivity] — XGBoost (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -22.3101 ± 18.3138
  MSE (mean±std):       10551.2594 ± 6409.4406
  MAE (mean±std):       94.1430 ± 27.3394
  RUL Score (mean±std): 15857044393911.6934 ± 350626805314865.0000
  Folds:                1040

Embargo=10 Results
  R2 (mean±std):        -46.0864 ± 62.9112
  MSE (mean±std):       11320.8032 ± 6621.3009
  MAE (mean±std):       98.6934 ± 27.5628
  RUL Score (mean±std): 1585704

In [20]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD003"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "xgb_only_pgts"

# Fixed model hyperparameters (XGBoost)
XGB_PARAMS = dict(
    objective="reg:squarederror",
    learning_rate=0.2,
    max_depth=4,
    n_estimators=200,
    subsample=1.0,
    colsample_bytree=0.8,
    random_state=42
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score = renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# XGBoost model builder
# ============================================
def build_xgb():
    return XGBRegressor(**XGB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_xgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_XGBoost")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — XGBoost
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
xgb_hold = build_xgb().fit(X_tr, y_tr)
p_hold = xgb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_XGBoost.csv")

# ============================================
# PGTS main (Embargo=10) — XGBoost
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — XGBoost
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_xgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_XGBoost")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — XGBoost
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — XGBoost
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — XGBoost]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main (opsiyonel olarak açılabilir)
# report_lines.append("\n[PGTS — XGBoost (Embargo=10)]")
# report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — XGBoost, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — XGBoost (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_XGBoost.txt", final_report)


Dataset=FD003 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — XGBoost]
  R2: 0.9779 | MSE: 216.6679 | MAE: 10.6474 | RUL Score: 27351.4890
[PGTS — Null Model (label permutation) — XGBoost, Embargo=10]
Results
  R2 (mean±std):        -0.3469 ± 0.3726
  MSE (mean±std):       7641.5246 ± 6298.4434
  MAE (mean±std):       68.9596 ± 25.0980
  RUL Score (mean±std): 63871589225696440.0000 ± 1009944183417075712.0000
  Folds:                400

[PGTS — Embargo Sensitivity] — XGBoost (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -15.3395 ± 9.6180
  MSE (mean±std):       13492.5845 ± 13000.7709
  MAE (mean±std):       101.1391 ± 42.4413
  RUL Score (mean±std): 12396752775116976128.0000 ± 231553471733017280512.0000
  Folds:                400

Embargo=10 Results
  R2 (mean±std):        -28.7058 ± 31.4559
  MSE (mean±std):       14370.5464 ± 13309.0770
  MAE (mean±std):       106.2103 ± 42.4538
  RUL Sc

In [21]:
# ============================================
# Configuration — choose CMAPSS subset once
# ============================================
# Set to "FD001", "FD002", "FD003", or "FD004"
DATASET_NAME = "FD004"

# Human-readable tag for the model family (used in output folder)
MODEL_TAG = "xgb_only_pgts"

# Fixed model hyperparameters (XGBoost)
XGB_PARAMS = dict(
    objective="reg:squarederror",
    learning_rate=0.2,
    max_depth=4,
    n_estimators=200,
    subsample=1.0,
    colsample_bytree=0.8,
    random_state=42
)

# PGTS config
W = 30        # Window length (cycles) ~ purge guidance
H = 1         # Horizon (steps ahead)
EMBARGO = 10  # cycles to skip after cut
N_SPLITS = 5  # number of cuts per engine

# ============================================
# Imports
# ============================================
import os
import time
import math
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor

# ============================================
# Output paths — single, unified directory
# ============================================
OUTPUT_DIR = Path(f"{MODEL_TAG}_{DATASET_NAME}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_txt(filename: str, text: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_DIR / Path(filename).name, "w", encoding="utf-8") as f:
        f.write(text)

def save_df_csv(df: pd.DataFrame, filename: str) -> None:
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DIR / Path(filename).name, index=False)

# ============================================
# I/O — paths derived from DATASET_NAME
# ============================================
# NASA CMAPSS public data
train_path = f"train_{DATASET_NAME}.txt"
test_path  = f"test_{DATASET_NAME}.txt"
rul_path   = f"RUL_{DATASET_NAME}.txt"

# ============================================
# Load
# ============================================
train = pd.read_csv(train_path, sep=r'\s+', header=None)
test  = pd.read_csv(test_path,  sep=r'\s+', header=None)
y_test_file = pd.read_csv(rul_path, sep=r'\s+', header=None)

# ============================================
# Columns
# ============================================
columns = [
    'id','cycle','setting1','setting2','setting3',
    's1','s2','s3','s4','s5','s6','s7','s8','s9',
    's10','s11','s12','s13','s14','s15','s16',
    's17','s18','s19','s20','s21'
]
train.columns = columns
test.columns  = columns

# ============================================
# Sort & clean
# ============================================
train.sort_values(['id','cycle'], inplace=True)
test.sort_values(['id','cycle'], inplace=True)
y_test_file.dropna(axis=1, inplace=True)

# ============================================
# Compute training RUL
# ============================================
max_cycle_train = train.groupby('id')['cycle'].max().reset_index()
max_cycle_train.columns = ['id','max_cycle']
train = train.merge(max_cycle_train, on='id', how='left')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Normalize features (fit on train only)
# ============================================
features = train.columns.difference(['id','cycle','RUL'])
assert len(features) == 24, f"Expected 24 features, got {len(features)}"
scaler = MinMaxScaler()
train_norm = pd.DataFrame(
    scaler.fit_transform(train[features]),
    columns=features, index=train.index
)
train = train[['id','cycle','RUL']].join(train_norm)

# ============================================
# Prepare test set (transform with train scaler)
# ============================================
test_norm = pd.DataFrame(
    scaler.transform(test[features]),
    columns=features, index=test.index
)
test = (
    test[test.columns.difference(features)]
    .join(test_norm)
    .reindex(columns=test.columns)
    .reset_index(drop=True)
)

# ============================================
# Compute test RUL from provided horizons
# ============================================
max_cycle_test = test.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id','max_cycle']
y_test_file.columns = ['collected_RUL']
y_test_file['id'] = y_test_file.index + 1
y_test_file['max_cycle'] = max_cycle_test['max_cycle'] + y_test_file['collected_RUL']
y_test_file.drop('collected_RUL', axis=1, inplace=True)
test = test.merge(y_test_file, on='id', how='left')
test['RUL'] = test['max_cycle'] - test['cycle']
test.drop('max_cycle', axis=1, inplace=True)

# ============================================
# Helpers: metrics (RUL Score = renamed PHM08)
# ============================================
def rul_score(y_true, y_pred):
    d = np.asarray(y_pred) - np.asarray(y_true)
    # same functional form as PHM08, but reported as "RUL Score"
    return float(np.sum(np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)))

def metrics_dict(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}RUL_Score": rul_score(y_true, y_pred)
    }

def summarize_table(rows, label):
    """rows: list of dicts with keys R2, MSE, MAE, RUL_Score"""
    df = pd.DataFrame(rows)
    if df.empty:
        return df, None
    summary = {
        "R2_mean": df["R2"].mean(),   "R2_std": df["R2"].std(),
        "MSE_mean": df["MSE"].mean(), "MSE_std": df["MSE"].std(),
        "MAE_mean": df["MAE"].mean(), "MAE_std": df["MAE"].std(),
        "RUL_Score_mean": df["RUL_Score"].mean(), "RUL_Score_std": df["RUL_Score"].std(),
        "n_folds": len(df)
    }
    save_df_csv(df, f"{label}_per_fold_{DATASET_NAME}.csv")
    save_df_csv(pd.DataFrame([summary]), f"{label}_summary_{DATASET_NAME}.csv")
    return df, summary

# ============================================
# XGBoost model builder
# ============================================
def build_xgb():
    return XGBRegressor(**XGB_PARAMS)

# ============================================
# Purged Group Time Series Split (PGTS)
# ============================================
ENGINE_COL = "id"
TIME_COL   = "cycle"
TARGET_COL = "RUL"
ALL_FEATURES = [c for c in train.columns if c not in [ENGINE_COL, TIME_COL, TARGET_COL]]

def pgts_splits_for_engine(df_engine, n_splits=N_SPLITS, window=W, horizon=H, embargo=EMBARGO):
    g = df_engine.sort_values(TIME_COL).reset_index()
    T = len(g)
    if T <= (window + horizon + 1):
        return []
    # choose cut points excluding edges
    cuts = np.linspace(window + horizon, T - horizon, num=n_splits+1, dtype=int)[1:]
    splits = []
    for cut in cuts:
        train_end  = max(window, cut - (window - 1))  # purge ~ W-1
        test_start = min(T - horizon, cut + embargo)
        if test_start <= train_end or test_start >= T - horizon:
            continue
        tr_idx = g.loc[:train_end-1, "index"].values
        te_idx = g.loc[test_start:T - horizon - 1, "index"].values
        if len(te_idx) == 0 or len(tr_idx) == 0:
            continue
        splits.append((tr_idx, te_idx))
    return splits

def run_pgts(df_all, features, embargo=EMBARGO, label="PGTS"):
    rows = []
    for _, df_e in df_all.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_all.loc[tr_idx, features]; y_tr = df_all.loc[tr_idx, TARGET_COL]
            X_te = df_all.loc[te_idx, features]; y_te = df_all.loc[te_idx, TARGET_COL]
            mdl = build_xgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_XGBoost")
    return summary

# ============================================
# Random 80/20 holdout (baseline) — XGBoost
# ============================================
X = train.drop('RUL', axis=1)
y = train['RUL']
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
xgb_hold = build_xgb().fit(X_tr, y_tr)
p_hold = xgb_hold.predict(X_va)
hold_summary = metrics_dict(y_va, p_hold)
save_df_csv(pd.DataFrame([hold_summary]), f"holdout_summary_{DATASET_NAME}_XGBoost.csv")

# ============================================
# PGTS main (Embargo=10) — XGBoost
# ============================================
df_all = train[[ENGINE_COL, TIME_COL, TARGET_COL] + ALL_FEATURES]
pgts_e10_summary = run_pgts(df_all, ALL_FEATURES, embargo=10, label="PGTS_E10")

# ============================================
# Null model (label permutation) under PGTS — XGBoost
# ============================================
def run_pgts_null(df_all, features, embargo=10, label="PGTS_NULL_E10", seed=42):
    rng = np.random.default_rng(seed)
    df_perm = df_all.copy()
    # permute labels within each engine independently
    df_perm[TARGET_COL] = (
        df_perm.groupby(ENGINE_COL)[TARGET_COL]
               .transform(lambda s: s.values[rng.permutation(len(s))])
    )
    rows = []
    for _, df_e in df_perm.groupby(ENGINE_COL, sort=False):
        for (tr_idx, te_idx) in pgts_splits_for_engine(df_e, embargo=embargo):
            X_tr = df_perm.loc[tr_idx, features]; y_tr = df_perm.loc[tr_idx, TARGET_COL]
            X_te = df_perm.loc[te_idx, features]; y_te = df_perm.loc[te_idx, TARGET_COL]
            mdl = build_xgb().fit(X_tr, y_tr)
            p = mdl.predict(X_te)
            rows.append(dict(R2=r2_score(y_te, p),
                             MSE=mean_squared_error(y_te, p),
                             MAE=mean_absolute_error(y_te, p),
                             RUL_Score=rul_score(y_te, p)))
    _, summary = summarize_table(rows, f"{label}_XGBoost")
    return summary

pgts_null_summary = run_pgts_null(df_all, ALL_FEATURES, embargo=10, label="PGTS_NULL_E10", seed=42)

# ============================================
# Embargo sensitivity (E=0 vs E=10) under PGTS — XGBoost
# ============================================
pgts_e0_summary  = run_pgts(df_all, ALL_FEATURES, embargo=0,  label="PGTS_E0")
pgts_e10_summary = pgts_e10_summary  # already computed above

# ============================================
# FINAL: compact report (print only summaries) — XGBoost
# ============================================
def fmt_summary(title, summary_dict):
    if summary_dict is None:
        return f"{title}: no valid folds."
    return (
        f"{title}\n"
        f"  R2 (mean±std):        {summary_dict['R2_mean']:.4f} ± {summary_dict['R2_std']:.4f}\n"
        f"  MSE (mean±std):       {summary_dict['MSE_mean']:.4f} ± {summary_dict['MSE_std']:.4f}\n"
        f"  MAE (mean±std):       {summary_dict['MAE_mean']:.4f} ± {summary_dict['MAE_std']:.4f}\n"
        f"  RUL Score (mean±std): {summary_dict['RUL_Score_mean']:.4f} ± {summary_dict['RUL_Score_std']:.4f}\n"
        f"  Folds:                {int(summary_dict['n_folds'])}\n"
    )

report_lines = []

# Feature count + config header
report_lines.append(f"Dataset={DATASET_NAME} | Feature count={len(ALL_FEATURES)}")
report_lines.append(f"Purged Group Time Series Split (PGTS): Window(W)={W}, Horizon(H)={H}, Embargo={EMBARGO}, Splits={N_SPLITS}")

# Holdout
report_lines.append("\n[Random 80/20 Split — XGBoost]")
report_lines.append(
    f"  R2: {hold_summary['R2']:.4f} | MSE: {hold_summary['MSE']:.4f} | MAE: {hold_summary['MAE']:.4f} | RUL Score: {hold_summary['RUL_Score']:.4f}"
)

# PGTS main (opsiyonel olarak açılabilir)
# report_lines.append("\n[PGTS — XGBoost (Embargo=10)]")
# report_lines.append(fmt_summary("Macro summary", pgts_e10_summary))

# Null model (PGTS)
report_lines.append("[PGTS — Null Model (label permutation) — XGBoost, Embargo=10]")
report_lines.append(fmt_summary("Results", pgts_null_summary))

# Embargo sensitivity
report_lines.append("[PGTS — Embargo Sensitivity] — XGBoost (Embargo 0 vs 10)")
report_lines.append(fmt_summary("Embargo=0 Results",  pgts_e0_summary))
report_lines.append(fmt_summary("Embargo=10 Results", pgts_e10_summary))

final_report = "\n".join(report_lines)
print(final_report)
save_txt(f"FINAL_REPORT_{DATASET_NAME}_XGBoost.txt", final_report)


Dataset=FD004 | Feature count=24
Purged Group Time Series Split (PGTS): Window(W)=30, Horizon(H)=1, Embargo=10, Splits=5

[Random 80/20 Split — XGBoost]
  R2: 0.9449 | MSE: 442.6934 | MAE: 16.1760 | RUL Score: 145513.5241
[PGTS — Null Model (label permutation) — XGBoost, Embargo=10]
Results
  R2 (mean±std):        -0.4550 ± 0.2849
  MSE (mean±std):       7761.1566 ± 5003.1169
  MAE (mean±std):       70.2308 ± 21.2966
  RUL Score (mean±std): 42698316085900280.0000 ± 886666804491315840.0000
  Folds:                996

[PGTS — Embargo Sensitivity] — XGBoost (Embargo 0 vs 10)
Embargo=0 Results
  R2 (mean±std):        -19.2626 ± 15.1279
  MSE (mean±std):       14566.3360 ± 11325.9022
  MAE (mean±std):       107.4866 ± 38.6735
  RUL Score (mean±std): 16163087539408936960.0000 ± 483784126309245648896.0000
  Folds:                996

Embargo=10 Results
  R2 (mean±std):        -36.2865 ± 48.2203
  MSE (mean±std):       15461.5895 ± 11610.9668
  MAE (mean±std):       112.2268 ± 38.7928
  RUL S